    Bachelor Thesis 2017 - Coffee - Climat - Soils data analysis
    
    HEIG-VD - University of Applied Sciences of Western Switzerland
    Route de Cheseaux 1, 1400 Yverdon-les-Bains, Switzerland
    
    International Center for Tropical Agriculture (CIAT)
    Headquarters and Regional Office for Latin America and the Caribbean, 
    Km 17 Recta Cali-Palmira │C.P. 763537 │ A.A. 6713  Cali, Colombia

    
    Student: Thibault Schowing
    Teacher: Carlos Andrés Peña
    
     


# Nettoyage ou modification spécifiques du dataset



## Création d'une version 2 et de datasets customs

Passage de la version 1 à la version 2 du set de données

Modification au besoin, ajout / suppression / modification des colonnes
Suppression des doublons
Pas de tolérence aux données manquantes

In [1]:
import pandas as pd
from datetime import *
import numpy as np
from datetime import datetime

def classCoffee(points):
    if points >= 90:
        # Outstanding
        return 1
    if points <90 and points >= 85:
        # Excellent
        return 2
    if points <85 and points >= 80:
        # Very Good
        return 3 
    else:
        return 4



# Transofrm orientation in categorical
# Normal = convert to category
# Not normal = return the same degrees

def convert_orientation(deg, normal = False):
    if(not normal):
        if deg == 0:
            return np.nan
        else:
            return deg
        
    if ((deg>0 and deg<22.5) or (deg > 337.5)):
        return "North"
    if (deg > 22.5 and deg <= 67.5):
        return "Northeast"
    if (deg > 67.5 and deg <= 112.5):
        return "East"
    if (deg > 112.5 and deg <= 157.5):
        return "Southeast"
    if (deg > 157.5 and deg <= 202.5):
        return "South"
    if (deg > 202.5 and deg <= 247.5):
        return "Southwest"
    if (deg > 247.5 and deg <= 292.5):
        return "West"
    if (deg > 292.5 and deg <= 337.5):
        return "Northwest"
    else:
        return np.nan

def convert_slope(slope):
    if slope == 0:
        return np.nan
    else:
        return slope

def convert(lum):
    if lum == "Sol":
        return 2
    elif lum == "Semi Sombra":
        return 1
    elif lum == "Sombra":
        return 0
    else:
        return 0



def days_between(d1, d2):
    d1 = datetime.strptime(d1, "%Y-%m-%d %H:%M:%S")
    d2 = datetime.strptime(d2, "%Y-%m-%d %H:%M:%S")
    return abs((d2 - d1).days)

In [2]:
# Read the CSV file (utf-8 encoded)
df = pd.read_csv('./data/DataRisaralda_v1_utf-8.csv', header=0, sep=',', index_col=0, parse_dates=False, encoding='utf-8')
print(df.shape)




(975, 139)


In [3]:

# Drop useless ok non-clean columns

try:
    df.drop('Cedula', axis=1, inplace=True)
except:
    print("No such column")
    

try:
    df.drop('Fecha Recpt', axis=1, inplace=True)
except:
    print("No such column")
    
    
try:
    df.drop('Factor de Rendimiento', axis=1, inplace=True)
except:
    print("No such column")    

# Not complete
try:
    df.drop('SICA', axis=1, inplace=True)
except:
    print("No such column") 

    
try:
    df.drop('Quaker (Nb Granos)', axis=1, inplace=True)
except:
    print("No such column") 
try:
    df.drop('Fluorescentes (Nb Granos)', axis=1, inplace=True)
except:
    print("No such column") 



    

try:
    df.drop('Malla 17-18', axis=1, inplace=True)
except:
    print("No such column") 
try:
    df.drop('Malla 14-16', axis=1, inplace=True)
except:
    print("No such column") 
try:
    df.drop('Malla <13', axis=1, inplace=True)
except:
    print("No such column") 



try:
    df.drop('REPRESENTA', axis=1, inplace=True)
except:
    print("No such column")
    
    
# Rename columns (mistakes and spaces removal) - other renaming follows...
# 
df.rename(columns={'EPSG:3116_X': 'EPSG:4326_X', 'EPSG:3116_Y': 'EPSG:4326_Y'}, inplace=True)
df.rename(columns={'CENT_X': 'EPSG:3116_X', 'CENT_Y': 'EPSG:3116_Y'}, inplace=True)
df.rename(columns={'LUMINOSID': 'Luminosidad'}, inplace=True)




# Format entries

for index, row in df.iterrows():
    date = datetime.strptime(str(row['Fecha Analysis']), '%Y-%m-%d %H:%M:%S')
    df.set_value(index,'Fecha Analysis', date)




# Print the different styles of luminosity

print df.Luminosidad.unique()

# Conversion in numerical values
for index, row in df.iterrows():
    df.set_value(index, 'Luminosidad', convert(row['Luminosidad']))
    

[u'Sol' u'Semi Sombra' u'Sombra']


In [4]:
cols = ['Malla 17-18','Malla 14-16','Malla <13','Negros y vinagres','Broca','Broca de punto','Veteado','Mordido',
         'Inmaduro','Flojo','Sobresecado','Arrugado','Aplastado','Cristalizado','Reposado','Granizo','Conchas',
         'Partido','Ambar','Defectos totales','Humedad (%)','Merma (%)','Aroma-Fragancia',
         'Acidez','Cuerpo','Sabor',
         'Sabor residual','Dulzor','Uniformidad','Balance','Taza limpia','Puntaje catador','Puntaje Total',
         'prec1','prec2','prec3','prec4','prec5','prec6','prec7','prec8','prec9','prec10','tmin1','tmin2','tmin3','tmin4','tmin5',
         'tmin6','tmin7','tmin8','tmin9','tmin10','tmax1','tmax2','tmax3','tmax4','tmax5','tmax6','tmax7','tmax8','tmax9','tmax10',
         'tmean1','tmean2','tmean3','tmean4','tmean5','tmean6','tmean7','tmean8','tmean9','tmean10','dtr1','dtr2','dtr3','dtr4',
         'dtr5','dtr6','dtr7','dtr8','dtr9','dtr10']
defectos = ['Negros y vinagres','Broca','Broca de punto','Veteado','Mordido',
         'Inmaduro','Flojo','Sobresecado','Arrugado','Aplastado','Cristalizado','Reposado','Granizo','Conchas',
         'Partido','Ambar']



## Ajout des moyennes sur les 10 mois de mesures

In [5]:
precs = ['prec1','prec2','prec3','prec4','prec5','prec6','prec7','prec8','prec9','prec10']
tmins = ['tmin1','tmin2','tmin3','tmin4','tmin5','tmin6','tmin7','tmin8','tmin9','tmin10']
tmaxs = ['tmax1','tmax2','tmax3','tmax4','tmax5','tmax6','tmax7','tmax8','tmax9','tmax10']
tmeans = ['tmean1','tmean2','tmean3','tmean4','tmean5','tmean6','tmean7','tmean8','tmean9','tmean10']
dtrs = ['dtr1','dtr2','dtr3','dtr4','dtr5','dtr6','dtr7','dtr8','dtr9','dtr10']

# Moyenne sur les 10 mois de mesures

df['PrecTotalAvg'] = 0.
df['TminTotalAvg'] = 0.
df['TmaxTotalAvg'] = 0.
df['TmeanTotalAvg'] = 0.
df['DtrTotalAvg'] = 0.

df['PrecTotal'] = 0.
df['TminTotal'] = 0.
df['TmaxTotal'] = 0.
df['TmeanTotal'] = 0.
df['DtrTotal'] = 0.


for index, row in df.iterrows():
    
    df.set_value(index,'PrecTotalAvg', row[precs].sum() / len(precs))
    df.set_value(index,'TminTotalAvg', row[tmins].sum() / len(precs))
    df.set_value(index,'TmaxTotalAvg', row[tmaxs].sum() / len(precs))
    df.set_value(index,'TmeanTotalAvg', row[tmeans].sum() / len(precs))
    df.set_value(index,'DtrTotalAvg', row[dtrs].sum() / len(precs))
    
    df.set_value(index,'PrecTotal', row[precs].sum())
    df.set_value(index,'TminTotal', row[tmins].sum())
    df.set_value(index,'TmaxTotal', row[tmaxs].sum())
    df.set_value(index,'TmeanTotal', row[tmeans].sum())
    df.set_value(index,'DtrTotal', row[dtrs].sum())


## Correction semi-manuelle des erreurs sur les points

In [6]:
catacionpts = ['Aroma-Fragancia','Acidez','Cuerpo','Sabor residual',
               'Sabor','Dulzor','Uniformidad','Balance','Taza limpia','Puntaje catador']

for index, row in df.iterrows():
        if row['Puntaje Total'] > 100:
            print index, row['Puntaje Total']
            df.set_value(index, 'Puntaje catador', 6.9)
            print row[catacionpts].sum()
            
            df.set_value(index, 'Puntaje Total', row[catacionpts].sum())

for index, row in df.iterrows():
        if row['Puntaje Total'] > 100:
            print index, row['Puntaje Total']

for index, row in df.iterrows():
        if np.isnan(row['Acidez']) or np.isnan(row['Puntaje Total']):
            df.set_value(index, catacionpts, 0)
            df.set_value(index, 'Puntaje Total', 0)
        if np.isnan(row['Defectos totales']):
            df.set_value(index, 'Defectos totales', row[defectos].sum())
            


## Convertion SICA

In [7]:
print df['SICA 1'].dtype

df['SICA 1'] = df['SICA 1'].astype(np.int64)
df['SICA 1'] = df['SICA 1'].astype(str)
print df['SICA 1'].dtype



float64
object


## Suppression des doublons

Basé sur la date de test et le numéro SICA

In [8]:
df = df.drop_duplicates(subset=['Fecha Analysis', 'SICA 1'], keep='first', inplace=False)
df = df.reset_index(drop=True)
print(df.shape)

(767, 134)


## Suppression des dégustations avec même SICA à moins de 3 mois d'écart

    Voir: analyse de l'occurence des cafés dans 2_CoffeDataAnalysis


In [9]:
# Certains cafés on été testés plusieurs fois, parfois même 2 fois dans la même année mais à plusieurs mois d'intervale. 

dOccurences = df['SICA 1'].value_counts().to_dict() #.to_frame(name='Occurence')

dfOccurences = pd.DataFrame.from_dict(dOccurences, orient='index')

# Ajout d'une colonne contenant le nombre d'occurence

df['Occurence'] = 0

# Fonctionne pas dans datacleaning...?
#df['SICA 1'] = df['SICA 1'].astype(str)

for index, row in df.iterrows():
    df.set_value(index, 'Occurence', dOccurences.get(str(row['SICA 1'])))

print df['Occurence'].value_counts()


#On prend ceux qui ont deux dates trop proches (pour 3, voir plus tard)
df_1 = df[df['Occurence'] == 2]

fields = ['SICA 1','Fecha Analysis','year']




df_1 = df_1[fields].sort_values(['SICA 1'])

1    631
2    118
3     18
Name: Occurence, dtype: int64


In [10]:
item = 1
oldDate = 1
todel = []

for index, row in df_1.iterrows():
    
    if item == 1:
        item = 2
        firstDate = row['Fecha Analysis']
        firstsica = row['SICA 1']
        continue
    if item == 2:
        secondDate = row['Fecha Analysis']
        secondsica = row['SICA 1']
        space = days_between(str(firstDate), str(secondDate))
        if space < 90:
            todel.append(index)
            print("Sica1 {} - Sica2 {}".format(firstsica,secondsica))
            print("Date1 {} - Date2 {}".format(firstDate,secondDate))
        item = 1

print todel

Sica1 6604502461 - Sica2 6604502461
Date1 2016-07-22 00:00:00 - Date2 2016-07-17 00:00:00
Sica1 6608800780 - Sica2 6608800780
Date1 2016-07-22 00:00:00 - Date2 2016-05-24 00:00:00
Sica1 6608800799 - Sica2 6608800799
Date1 2016-07-25 00:00:00 - Date2 2016-09-14 00:00:00
Sica1 6608800899 - Sica2 6608800899
Date1 2016-07-22 00:00:00 - Date2 2016-07-24 00:00:00
Sica1 6608800959 - Sica2 6608800959
Date1 2016-07-24 00:00:00 - Date2 2016-07-21 00:00:00
Sica1 6617000764 - Sica2 6617000764
Date1 2016-05-16 00:00:00 - Date2 2016-07-22 00:00:00
Sica1 6638300445 - Sica2 6638300445
Date1 2016-06-23 00:00:00 - Date2 2016-06-02 00:00:00
Sica1 6644000388 - Sica2 6644000388
Date1 2016-07-22 00:00:00 - Date2 2016-06-23 00:00:00
Sica1 6644000596 - Sica2 6644000596
Date1 2016-07-22 00:00:00 - Date2 2016-05-21 00:00:00
Sica1 6644001113 - Sica2 6644001113
Date1 2016-07-22 00:00:00 - Date2 2016-05-16 00:00:00
Sica1 6645600841 - Sica2 6645600841
Date1 2011-12-07 00:00:00 - Date2 2011-12-29 00:00:00
Sica1 6659

In [11]:
df = df.drop(todel)
todel = []
df = df.reset_index(drop=True)
print df.shape

(746, 135)


Elimination manuelle des 6 cafés se répéttant 3x (il y en a 6) - si possible en laisser 2, toujours avec la règle des 3 mois d'écart sinon éliminer le plus ancien. 

In [12]:
df_2 = df[df['Occurence'] == 3].sort_values(['SICA 1'])
cols = ['SICA 1','Fecha Analysis']
df_2[cols]

,SICA 1,Fecha Analysis
729,6608800200,2016-09-14 00:00:00
682,6608800200,2016-06-04 00:00:00
379,6608800200,2016-07-22 00:00:00
136,6608800488,2011-11-30 00:00:00
367,6608800488,2016-07-24 00:00:00
382,6608800488,2016-07-22 00:00:00
352,6608801616,2016-07-24 00:00:00
730,6608801616,2016-09-14 00:00:00
681,6608801616,2016-05-16 00:00:00
219,6644001444,2011-12-23 00:00:00


In [13]:
toDel = [379,382,352,680,399]
df = df.drop(todel)
todel = []
df = df.reset_index(drop=True)
print df.shape

(746, 135)


## Compact texture

In [14]:
francos = ["franco_L1","franco_L2","franco_L3"]
arcillosos = ["arcilloso_L1","arcilloso_L2","arcilloso_L3"]
limosos = ["limoso_L1","limoso_L2","limoso_L3"]
arenosos = ["arenoso_L1","arenoso_L2","arenoso_L3"]
cascajosos = ["cascajoso_L1","cascajoso_L2","cascajoso_L3"]

# Ajout de colonnes pour résumer la texture du sol.
df["Franco"] = 0
df["Arcilloso"] = 0
df["Limoso"] = 0
df["Arenoso"] = 0
df["Cascajoso"] = 0


for index, row in df.iterrows():
    df.set_value(index, 'Franco', row[francos].sum())
    df.set_value(index, 'Arcilloso', row[arcillosos].sum())
    df.set_value(index, 'Limoso', row[limosos].sum())
    df.set_value(index, 'Arenoso', row[arenosos].sum())
    df.set_value(index, 'Cascajoso', row[cascajosos].sum())

sols = ["franco_L1","arcilloso_L1","limoso_L1","arenoso_L1","cascajoso_L1",
        "franco_L2","arcilloso_L2","limoso_L2","arenoso_L2","cascajoso_L2",
        "franco_L3","arcilloso_L3","limoso_L3","arenoso_L3","cascajoso_L3"]


for col in sols:
    df.drop(col, axis=1, inplace=True)






## Ajout d'une catégorie pour chaque cafés

### Liste des catégories

|Total Score|       Quality Classification||
|---|---|---|
|90-100	|          Outstanding|	             Specialty|
|85-89.99	|      Excellent|	             Specialty|
|80-84.99|	      Very Good|	             Specialty|
|<80.0	|          Below Specialty Quality|	 Not Specialty|


http://www.scaa.org/?page=resources&d=cupping-protocols

In [15]:
# Catégorie de cafés entre 1 )plus de 90 points et 4 (moins de 80)
df['Category'] = 0

for index, row in df.iterrows():
    df.set_value(index, 'Category', classCoffee(row['Puntaje Total']))

## Rename columns

In [16]:
df.rename(columns={'Defectos totales': 'DefectosTotales'}, inplace=True)
df.rename(columns={'SICA 1': 'SICA'}, inplace=True)
df.rename(columns={'LUMINOSID': 'Luminosidad'}, inplace=True)
df.rename(columns={'Defectos totales': 'DefectosTotales'}, inplace=True)
df.rename(columns={'Puntaje Total': 'PuntajeTotal'}, inplace=True)

df.rename(columns={'Fecha Analysis': 'FechaAnalysis'}, inplace=True)

df.rename(columns={'Negros y vinagres': 'NegrosYVinagres'}, inplace=True)
df.rename(columns={'Broca de punto': 'BrocaDePunto'}, inplace=True)
df.rename(columns={'Defectos totales': 'DefectosTotales'}, inplace=True)
df.rename(columns={'Humedad (%)': 'Humedad'}, inplace=True)
df.rename(columns={'Merma (%)': 'Merma'}, inplace=True)
df.rename(columns={'Sabor residual': 'SaborResidual'}, inplace=True)
df.rename(columns={'Taza limpia': 'TazaLimpia'}, inplace=True)
df.rename(columns={'Puntaje catador': 'PuntajeCatador'}, inplace=True)
df.rename(columns={'Puntaje Total': 'PuntajeTotal'}, inplace=True)
df.rename(columns={'SICA 1': 'SICA'}, inplace=True)
df.rename(columns={'CEDULA 1': 'Cedula'}, inplace=True)

df.rename(columns={'VARIEDAD': 'Variedad'}, inplace=True)

df.rename(columns={'Taza 1': 'Taza1'}, inplace=True)
df.rename(columns={'Taza 2': 'Taza2'}, inplace=True)
df.rename(columns={'Taza 3': 'Taza3'}, inplace=True)
df.rename(columns={'Taza 4': 'Taza4'}, inplace=True)
df.rename(columns={'Taza 5': 'Taza5'}, inplace=True)






## Rename variety

In [17]:
df['Variedad'] = df['Variedad'].replace({'T?PICA': 'TIPICA'})

## Ajout de Slope et Orientation
Réalisé dans le set compact


In [18]:
## Adding slope and orientation 

 
df_temp = pd.read_csv('./data/Slope_Orientation/Risaralda_orientation_slope.csv', header=0, sep=',', index_col=0, parse_dates=False, encoding='utf-8')
#df_temp = df_temp.drop('FID', 1)


cols = ["SICA_1","ris_aspect","ris_slope"]
df_temp = df_temp[cols]

df_temp.rename(columns={'SICA_1': 'SICA'}, inplace=True)
df_temp.rename(columns={'ris_aspect': 'OrientationNum'}, inplace=True)
df_temp.rename(columns={'ris_slope': 'Slope'}, inplace=True)

df_temp['SICA'] = df_temp['SICA'].round(0).astype(np.int64)
df_temp['SICA'] = df_temp['SICA'].astype(str)

#print df_temp

print df_temp.shape
print df.shape


d_orient = dict(zip(df_temp.SICA, df_temp.OrientationNum))
d_slope = dict(zip(df_temp.SICA, df_temp.Slope))


#print d_slope.get('6607500996')

df['OrientationNum'] = 0.
df['Slope'] = 0.

for index, row in df.iterrows():
    df.set_value(index, 'OrientationNum', d_orient.get(row['SICA']))
    df.set_value(index, 'Slope', d_slope.get(row['SICA']))
print df.shape


(746, 3)
(746, 126)
(746, 128)


In [19]:
for c in df.columns:
    print c

Municipio
Vereda
Finca
Cedula
SICA
Variedad
EPSG:3116_X
EPSG:3116_Y
EPSG:4326_Y
EPSG:4326_X
ASNM
Luminosidad
FechaAnalysis
NegrosYVinagres
Broca
BrocaDePunto
Veteado
Mordido
Inmaduro
Flojo
Sobresecado
Arrugado
Aplastado
Cristalizado
Reposado
Granizo
Conchas
Partido
Ambar
DefectosTotales
Pergamino
Almendra
AlmendraTotal
AlmendraSana
UV
Olor
Humedad
Merma
Aroma-Fragancia
Acidez
Cuerpo
Sabor
SaborResidual
Dulzor
Uniformidad
Balance
TazaLimpia
PuntajeCatador
PuntajeTotal
Taza1
Taza2
Taza3
Taza4
Taza5
prec1
prec2
prec3
prec4
prec5
prec6
prec7
prec8
prec9
prec10
tmin1
tmin2
tmin3
tmin4
tmin5
tmin6
tmin7
tmin8
tmin9
tmin10
tmax1
tmax2
tmax3
tmax4
tmax5
tmax6
tmax7
tmax8
tmax9
tmax10
tmean1
tmean2
tmean3
tmean4
tmean5
tmean6
tmean7
tmean8
tmean9
tmean10
dtr1
dtr2
dtr3
dtr4
dtr5
dtr6
dtr7
dtr8
dtr9
dtr10
year
PrecTotalAvg
TminTotalAvg
TmaxTotalAvg
TmeanTotalAvg
DtrTotalAvg
Soil Profile
Unidad_c_1
pH_avg
org_avg
PrecTotal
TminTotal
TmaxTotal
TmeanTotal
DtrTotal
Occurence
Franco
Arcilloso
Limoso


## Ajout de données chimiques de sol

Après la visite à Felipe, on peut avoir plus d'informations chimiques sur le sol. Si le temps le permet, on les ajoute. Les analyses peuvent être refaites rapidement si les scripts sont bien fait. 

UPDATE: La plateforme données et incomplète et ne permet pas l'extraction automatisée de données. 

TODO: réaliser une classe pour le set V2 permettant de get des informations à partir du SICA

## Mise en ordre des colonnes

In [20]:
cols = [
"SICA"
,"Cedula"
,"Municipio"
,"Vereda"
,"Finca"
,"EPSG:3116_X","EPSG:3116_Y","EPSG:4326_Y","EPSG:4326_X"
,"Variedad"
,"FechaAnalysis"
,"year"
,"Occurence"
,"UV","Olor","Humedad","Merma","Pergamino","Almendra","AlmendraTotal","AlmendraSana"
,"NegrosYVinagres","Broca","BrocaDePunto","Veteado","Mordido","Inmaduro","Flojo","Sobresecado","Arrugado"
,"Aplastado","Cristalizado","Reposado","Granizo","Conchas","Partido","Ambar","DefectosTotales"
,"ASNM"
,"Luminosidad"
,"prec1","prec2","prec3","prec4","prec5","prec6","prec7","prec8","prec9","prec10"
,"tmin1","tmin2","tmin3","tmin4","tmin5","tmin6","tmin7","tmin8","tmin9","tmin10"
,"tmax1","tmax2","tmax3","tmax4","tmax5","tmax6","tmax7","tmax8","tmax9","tmax10"
,"tmean1","tmean2","tmean3","tmean4","tmean5","tmean6","tmean7","tmean8","tmean9","tmean10"
,"dtr1","dtr2","dtr3","dtr4","dtr5","dtr6","dtr7","dtr8","dtr9","dtr10"
,"PrecTotal","TminTotal","TmaxTotal","TmeanTotal","DtrTotal"
,"PrecTotalAvg","TminTotalAvg","TmaxTotalAvg","TmeanTotalAvg","DtrTotalAvg"
,"OrientationNum"
,"Slope"
,"Soil Profile"
,"Unidad_c_1"
,"pH_avg"
,"org_avg"
,"Franco"
,"Arcilloso"
,"Limoso"
,"Arenoso"
,"Cascajoso"
,"Taza1","Taza2","Taza3","Taza4","Taza5"
,"Aroma-Fragancia","Acidez","Cuerpo","Sabor","SaborResidual","Dulzor","Uniformidad","Balance","TazaLimpia","PuntajeCatador","PuntajeTotal"
,"Category"]

df = df[cols]

## Ecriture du dataset V2 

Contient toutes les données (nom de la ferme (Finca) etc)

In [21]:
df.to_csv("./data/DataRisaralda_v2_utf-8.csv", sep=',', encoding='utf-8')
df.to_csv("./data/DataRisaralda_v2_iso-8859-1.csv", sep=',', encoding='iso-8859-1')

---------------------------------------------------

# Datasets customs

_______________________________________________

## Set SICA - GPS - Points Totaux

In [22]:
cols_GPS = ['SICA','EPSG:3116_X','EPSG:3116_Y','EPSG:4326_X','EPSG:4326_Y','PuntajeTotal']
df_coordinates_pts = df[cols_GPS]

df_coordinates_pts.to_csv("./data/Coordinates_Points_Risaralda_utf-8.csv", sep=',', encoding='utf-8')
df_coordinates_pts.to_csv("./data/Coordinates_Points_Risaralda_iso-8859-1.csv", sep=',', encoding='iso-8859-1')

## Set SICA - GPS - Catégories

In [23]:
cols_GPS = ['SICA','EPSG:3116_X','EPSG:3116_Y','EPSG:4326_X','EPSG:4326_Y','Category']
df_coordinates_cat = df[cols_GPS]

df_coordinates_cat.to_csv("./data/Coordinates_Category_Risaralda_utf-8.csv", sep=',', encoding='utf-8')
df_coordinates_cat.to_csv("./data/Coordinates_Category_Risaralda_iso-8859-1.csv", sep=',', encoding='iso-8859-1')

## Dataset numérique complet
    sans données manquantes au niveau des colonnes !

In [34]:
print("Création d'un set numérique... Suppression des champs non numérique et non utiles")

cols_num = [
"SICA"
,"year"
,"DefectosTotales"
,"ASNM"
,"Luminosidad"
,"prec1","prec2","prec3","prec4","prec5","prec6","prec7","prec8","prec9","prec10"
,"tmin1","tmin2","tmin3","tmin4","tmin5","tmin6","tmin7","tmin8","tmin9","tmin10"
,"tmax1","tmax2","tmax3","tmax4","tmax5","tmax6","tmax7","tmax8","tmax9","tmax10"
,"tmean1","tmean2","tmean3","tmean4","tmean5","tmean6","tmean7","tmean8","tmean9","tmean10"
,"dtr1","dtr2","dtr3","dtr4","dtr5","dtr6","dtr7","dtr8","dtr9","dtr10"
,"PrecTotal","TminTotal","TmaxTotal","TmeanTotal","DtrTotal"
,"PrecTotalAvg","TminTotalAvg","TmaxTotalAvg","TmeanTotalAvg","DtrTotalAvg"
,"OrientationNum"
,"Slope"
,"pH_avg"
,"org_avg"
,"Franco"
,"Arcilloso"
,"Limoso"
,"Arenoso"
,"Cascajoso"
,"Aroma-Fragancia","Acidez","Cuerpo","Sabor","SaborResidual","Dulzor","Uniformidad","Balance","TazaLimpia","PuntajeCatador","PuntajeTotal"
,"Category"]


df_num = df[cols_num]

print("\nSuppression des colonnes avec des éventuelles données manquantes....")
df_num_complete = df_num.dropna(axis=1, how='any')

print("\nDataset final avec shape de {}".format(df_num_complete.shape))

df_num_complete.to_csv("./data/DataRisaralda_v2Numeric_Complete_utf-8.csv", sep=',', encoding='utf-8')
df_num_complete.to_csv("./data/DataRisaralda_v2Numeric_Complete_iso-8859-1.csv", sep=',', encoding='iso-8859-1')


print("\n\nFichiers DataRisaralda_v2Numeric_Complete_utf-8.csv et DataRisaralda_v2Numeric_Complete_iso-8859-1.csv créés")

Création d'un set numérique... Suppression des champs non numérique et non utiles

Suppression des colonnes avec des éventuelles données manquantes....

Dataset final avec shape de (746, 86)


Fichiers DataRisaralda_v2Numeric_Complete_utf-8.csv et DataRisaralda_v2Numeric_Complete_iso-8859-1.csv créés


## Datasets pour R

Inclusion d'une colonne de catégorie (variété ou année par exemple)

## Sortie: PUNTAJE TOTAL

In [26]:
print("Création d'un set pour R avec Puntaje Total comme sortie")

cols_R_Total = [
"SICA"
,"year"
,"DefectosTotales"
,"ASNM"
,"Luminosidad"
,"prec1","prec2","prec3","prec4","prec5","prec6","prec7","prec8","prec9","prec10"
,"tmin1","tmin2","tmin3","tmin4","tmin5","tmin6","tmin7","tmin8","tmin9","tmin10"
,"tmax1","tmax2","tmax3","tmax4","tmax5","tmax6","tmax7","tmax8","tmax9","tmax10"
,"tmean1","tmean2","tmean3","tmean4","tmean5","tmean6","tmean7","tmean8","tmean9","tmean10"
,"dtr1","dtr2","dtr3","dtr4","dtr5","dtr6","dtr7","dtr8","dtr9","dtr10"
,"PrecTotal","TminTotal","TmaxTotal","TmeanTotal","DtrTotal"
,"PrecTotalAvg","TminTotalAvg","TmaxTotalAvg","TmeanTotalAvg","DtrTotalAvg"
,"OrientationNum"
,"Slope"
,"pH_avg"
,"org_avg"
,"Franco"
,"Arcilloso"
,"Limoso"
,"Arenoso"
,"Cascajoso"
,"Variedad"
,"PuntajeTotal"
]


df_R_Total = df[cols_R_Total]

# le script doit avoir une colonne appelée (peu importe le nom) de catégories. A spécifier dans R. Ici c'est "Variedad"

print("\nSuppression des colonnes avec des éventuelles données manquantes....")
df_R_Total = df_R_Total.dropna(axis=1, how='any')

print("\nDataset pour R avec shape de {}".format(df_R_Total.shape))

df_R_Total.to_csv("./data/DataRisaralda_v2_R_Total_utf-8.csv", sep=',', encoding='utf-8')

print("\n\nFichier DataRisaralda_v2_R_Total_utf-8.csv créé")

Création d'un set pour R

Suppression des colonnes avec des éventuelles données manquantes....


## Sortie: ACIDEZ

In [33]:
print("Création d'un set pour R avec Acidez comme sortie")

cols_R_Acidez = [
"SICA"
,"year"
,"DefectosTotales"
,"ASNM"
,"Luminosidad"
,"prec1","prec2","prec3","prec4","prec5","prec6","prec7","prec8","prec9","prec10"
,"tmin1","tmin2","tmin3","tmin4","tmin5","tmin6","tmin7","tmin8","tmin9","tmin10"
,"tmax1","tmax2","tmax3","tmax4","tmax5","tmax6","tmax7","tmax8","tmax9","tmax10"
,"tmean1","tmean2","tmean3","tmean4","tmean5","tmean6","tmean7","tmean8","tmean9","tmean10"
,"dtr1","dtr2","dtr3","dtr4","dtr5","dtr6","dtr7","dtr8","dtr9","dtr10"
,"PrecTotal","TminTotal","TmaxTotal","TmeanTotal","DtrTotal"
,"PrecTotalAvg","TminTotalAvg","TmaxTotalAvg","TmeanTotalAvg","DtrTotalAvg"
,"OrientationNum"
,"Slope"
,"pH_avg"
,"org_avg"
,"Franco"
,"Arcilloso"
,"Limoso"
,"Arenoso"
,"Cascajoso"
,"Variedad"
,"Acidez"
]


df_R_Acidez = df[cols_R_Acidez]

# le script doit avoir une colonne appelée (peu importe le nom) de catégories. A spécifier dans R. Ici c'est "Variedad"

print("\nSuppression des colonnes avec des éventuelles données manquantes....")
df_R_Acidez = df_R_Acidez.dropna(axis=1, how='any')


print("\nDataset pour R avec shape de {}".format(df_R_Acidez.shape))

df_R_Acidez.to_csv("./data/DataRisaralda_v2_R_Acidez_utf-8.csv", sep=',', encoding='utf-8')

print("\n\nFichier DataRisaralda_v2_R_Acidez_utf-8.csv créé")

Création d'un set pour R avec Acidez comme sortie

Suppression des colonnes avec des éventuelles données manquantes....

Dataset pour R avec shape de (746, 76)


Fichier DataRisaralda_v2_R_Acidez_utf-8.csv créé


## SORTIE: DULZOR

In [32]:
print("Création d'un set pour R avec Dulzor comme sortie")

cols_R_Dulzor = [
"SICA"
,"year"
,"DefectosTotales"
,"ASNM"
,"Luminosidad"
,"prec1","prec2","prec3","prec4","prec5","prec6","prec7","prec8","prec9","prec10"
,"tmin1","tmin2","tmin3","tmin4","tmin5","tmin6","tmin7","tmin8","tmin9","tmin10"
,"tmax1","tmax2","tmax3","tmax4","tmax5","tmax6","tmax7","tmax8","tmax9","tmax10"
,"tmean1","tmean2","tmean3","tmean4","tmean5","tmean6","tmean7","tmean8","tmean9","tmean10"
,"dtr1","dtr2","dtr3","dtr4","dtr5","dtr6","dtr7","dtr8","dtr9","dtr10"
,"PrecTotal","TminTotal","TmaxTotal","TmeanTotal","DtrTotal"
,"PrecTotalAvg","TminTotalAvg","TmaxTotalAvg","TmeanTotalAvg","DtrTotalAvg"
,"OrientationNum"
,"Slope"
,"pH_avg"
,"org_avg"
,"Franco"
,"Arcilloso"
,"Limoso"
,"Arenoso"
,"Cascajoso"
,"Variedad"
,"Dulzor"
]


df_R_Dulzor = df[cols_R_Dulzor]

# le script doit avoir une colonne appelée (peu importe le nom) de catégories. A spécifier dans R. Ici c'est "Variedad"

print("\nSuppression des colonnes avec des éventuelles données manquantes....")
df_R_Dulzor = df_R_Dulzor.dropna(axis=1, how='any')


print("\nDataset pour R avec shape de {}".format(df_R_Dulzor.shape))

df_R_Dulzor.to_csv("./data/DataRisaralda_v2_R_Dulzor_utf-8.csv", sep=',', encoding='utf-8')

print("\n\nFichier DataRisaralda_v2_R_Dulzor_utf-8.csv créé")

Création d'un set pour R avec Dulzor comme sortie

Suppression des colonnes avec des éventuelles données manquantes....

Dataset pour R avec shape de (746, 76)


Fichier DataRisaralda_v2_R_Dulzor_utf-8.csv créé


## PCA - Entrées simplifiées

In [37]:
cols_R_PCA = [
"year"
,"DefectosTotales"
,"ASNM"
,"Luminosidad"
,"PrecTotalAvg","TminTotalAvg","TmaxTotalAvg","TmeanTotalAvg","DtrTotalAvg"
,"OrientationNum"
,"Slope"
,"pH_avg"
,"org_avg"
,"Franco"
,"Arcilloso"
,"Limoso"
,"Arenoso"
,"Cascajoso"
,"Aroma-Fragancia","Acidez","Cuerpo","Sabor","SaborResidual","Dulzor"
,"Uniformidad","Balance","TazaLimpia","PuntajeCatador","PuntajeTotal"
]

df_R_PCA = df[cols_R_PCA]

# le script doit avoir une colonne appelée (peu importe le nom) de catégories. A spécifier dans R. Ici c'est "Variedad"

print("\nSuppression des colonnes avec des éventuelles données manquantes....")
df_R_PCA = df_R_PCA.dropna(axis=1, how='any')


print("\nDataset pour R avec shape de {}".format(df_R_PCA.shape))

df_R_PCA.to_csv("./data/DataRisaralda_v2_R_PCA_utf-8.csv", sep=',', encoding='utf-8')

print("\n\nFichier DataRisaralda_v2_R_PCA_utf-8.csv créé")


Suppression des colonnes avec des éventuelles données manquantes....

Dataset pour R avec shape de (746, 29)


Fichier DataRisaralda_v2_R_PCA_utf-8.csv créé
